In [ ]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import PorterStemmer as stemmer
from tqdm.notebook import tqdm
import rule_book_functs as rbfuncts

In [ ]:
incidents_fn = "data/source/20220413_D1_Incidents.csv"
rule_book_fn = "data/rule_book.csv"

In [ ]:
rules = pd.read_csv(rule_book_fn)
incidents = pd.read_csv(incidents_fn, dtype=str)

In [ ]:
incidents.rename(columns={'IncidentNumber': 'incident_id'}, inplace=True)
incidents['text'] = (
        incidents['ShortDescription'].astype(str).fillna('') + ' ' + 
        incidents['FullDescription'].astype(str).fillna('') + ' ' + 
        incidents['ImmediateAction'].astype(str).fillna('')
).str.lower()

incidents = incidents[['incident_id', 'text']]

In [ ]:
#sample100 = incidents.sample(n=100)
#sample100.index = range(100)
#sample100.to_csv('sample100.csv', index=False)
sample100 = pd.read_csv('sample100.csv', dtype=str)

In [ ]:
#incidents.text[0]

In [ ]:
#test = incidents.text.head(1).apply(rbfuncts.lemmatize_text)
#print(list(test))

In [ ]:
#test = incidents.text.head(1).apply(rbfuncts.stem_text)
#print(list(test))

In [ ]:
#test_tokens = rbfuncts.tokenize(incidents.text[0])

In [ ]:
#chk_text = 'employee was grinding with a four-inch grinder on an iron support in the compressor building when he felt discomfort to his left eye.'
#pattern = '(\\bemployee (.*) discomfort (.*) eye\\b)'
#rbfuncts.check_presence(pattern, chk_text)

In [ ]:
#sents = sent_tokenize(incidents.text[0])
#for sent in sents:
#    if rbfuncts.check_presence(pattern, sent):
#        print(f'{rbfuncts.check_presence(pattern, sent)}: {sent}')

## Load Synonyms & Rules

In [ ]:
syn_csv = pd.read_csv('synonyms.csv')

# Load the dictionary of synonyms
syn_dict = {}
for r in range(len(syn_csv)):
    syn_toks = syn_csv.keywords[r].split(',')
    syn_dict.update({syn_csv.syn[r]:syn_toks})
    
rul_csv = pd.read_csv('rules.csv')

In [ ]:
finds_df = rbfuncts.rule_book_scan(sample100, syn_dict, rul_csv)

True

In [24]:
#finds_df

In [ ]:
strs = 'This is me. He tripped and fell from the ladder.'
#strs = 'during the scaffolding elements transporting for the assembly of scaffolding scaffolder slipped and fell due to the slippery floor'
test = pd.DataFrame({'incident_id':[1052], 'text':[strs]})
rbfuncts.rule_book_scan(test, syn_dict, rul_csv)

In [ ]:
print(not False)

In [ ]:
# How many incidents were classified
len(list(dict.fromkeys(finds_df[finds_df.finds_list == True].incid_nums)))

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='finds')

In [ ]:
rbfuncts.deepdive_results(sample100, incidents, finds_df, focus='misses')

In [ ]:
chk = 'the country gm and regional gm were on their way to their apartment when they were involved in a motor vehicle collision.'
#check_presence('hello(.*)vehicle(.*)collision', chk)
#check_presence('\bregional (.*) were (.*) way\b', chk)
#bool(re.search('\bregional (.*) were (.*) way\b', chk))
bool(re.search('involved(.*)motor(.*)collision', chk))

In [ ]:
org_string = "This is employee's hand"
pattern = r"'s"

# Replace all occurrences of character 's with an empty string
org_string = re.sub(pattern, '', org_string )
print(org_string)

In [9]:
def check_presence(pattern, string):
    if pattern:
        return bool(re.search(pattern, string))
    else:
        return False

In [26]:
rul_csv

,rule,keyword,syns,voids,span,shuffle
0,eye injury or irritation,eye,"worker_syns, pain_syns, eye_syns",NaN,8,True
1,eye injury or irritation,eye,"gen1_syns, pain_syns, eye_syns",NaN,8,True
2,back injury,back,"worker_syns, back_pain_syns, back_syns",NaN,8,True
3,slips and trips,-,"worker_syns, slip_syns, fall_syns",ladder,8,True
4,vehicle incident,-,"vehicle_syns, vehicle_move_syns, vehicle_syns",NaN,8,False
5,vehicle incident,-,"vehicle_syns, vehicle_move_syns, object_syns",NaN,8,True
6,vehicle incident,-,"involve_syns, vehicle_syns, collision_syns",NaN,8,False


In [17]:
x = 'debris(.*)eye(.*)employee'
pattern = x
check_presence(pattern, "This debris was in the eyes of the employee's hand")

True

In [2]:
rbfuncts.rule_book_scan(test, syn_dict, rul_csv)

Checking rule 4 of 7 (slips and trips)


  0%|          | 0/1 [00:00<?, ?it/s]

,incid_nums,finds_pats,finds_list,incid_cats


In [35]:
test = pd.DataFrame({'id':[1, 2, 2, 2, 3], 'cats':['apple', 'apple', 'apple', 'orange', 'kiwi']})
test

,id,cats
0,1,apple
1,2,apple
2,2,apple
3,2,orange
4,3,kiwi


In [36]:
test.groupby(['id'])['cats'].apply(', '.join).reset_index()

,id,cats
0,1,apple
1,2,"apple, apple, orange"
2,3,kiwi


In [37]:
'apple, apple, orange'.split(', ')

['apple', 'apple', 'orange']

In [38]:
list(dict.fromkeys('apple, apple, orange'.split(', ')))

['apple', 'orange']

In [41]:
", ".join(list(dict.fromkeys('apple, apple, orange'.split(', '))))

'apple, orange'

In [42]:
def remove_dups(ent):
    split_ent = ent.split(', ')
    uniqe_ent = list(dict.fromkeys(split_ent))
    rjoin_ent = ", ".join(uniqe_ent)
    return(rjoin_ent)

In [43]:
remove_dups('apple, apple, orange')

'apple, orange'

In [48]:
test = test.groupby(['id'])['cats'].apply(', '.join).reset_index()
test['cats'] = test['cats'].apply(lambda x: remove_dups(x))

In [49]:
test

,id,cats
0,1,apple
1,2,"apple, orange"
2,3,kiwi
